

-------

# **`Data Ingestion`**

-------




In [1]:
import os
%pwd

'e:\\Practice python\\Chest Cancer Classification Using MLflow and DVC\\research'

In [2]:
os.chdir("../")

In [3]:

%pwd

'e:\\Practice python\\Chest Cancer Classification Using MLflow and DVC'

In [4]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.chest_cancer_classifier import *
from src.chest_cancer_classifier.utils.common_functions import read_yaml, create_directories

In [6]:
from src.chest_cancer_classifier.constants import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [7]:
import os
import zipfile
import gdown
from src.chest_cancer_classifier import logger
from src.chest_cancer_classifier.utils.common_functions import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-10 20:52:56,748: INFO: common_functions: YAML file 'config\config.yaml' loaded successfully]
[2024-11-10 20:52:56,765: INFO: common_functions: YAML file 'params.yaml' loaded successfully]
[2024-11-10 20:52:56,769: INFO: common_functions: Directory created at: artifacts]
[2024-11-10 20:52:56,772: INFO: common_functions: Directory created at: artifacts/data_ingestion]
[2024-11-10 20:52:56,776: INFO: 3737630271: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


ConnectTimeout: HTTPSConnectionPool(host='drive.google.com', port=443): Max retries exceeded with url: /uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000293FDBA33D0>, 'Connection to drive.google.com timed out. (connect timeout=None)'))

-----